In [19]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from code_vi.system import OpticsManager
from code_vi.elements import OpticalElement
from code_vi.ray_trace import RayTracer
from code_vi.visualization import Draw
from code_vi import optimization

# 1. Initialize
manager = OpticsManager()

lens1 = OpticalElement(
    name="Lens 1", optic_type="Lens",
    x_center=12.7, y_center=250.42, 
    orientation_angle=90.0,         
    clear_aperture=22.86, diameter=25.4, 
    center_thickness=2.4,           
    R1=np.inf, R2=-350.67,            
    k2=0,                    
    coeffs2=[], 
    material="ZnSe"
)

# LENS 2: Convex surface faces Lens 1.
# Center is placed at 753.34 mm to maintain exactly 2*EFL (500.0 mm) between principal planes
lens2 = OpticalElement(
    name="Lens 2", optic_type="Lens",
    x_center=12.7, y_center=753.34,   
    orientation_angle=90.0,          
    clear_aperture=22.86, diameter=25.4, 
    center_thickness=2.4,           
    R1=350.67, R2=np.inf,            
    k1=0,                    
    coeffs1=[], 
    material="ZnSe"
)

# --- EXACT THICK-LENS FOCAL PLANE ---
# Image forms exactly BFL (247.92 mm) past Lens 2's flat back surface (755.84 mm)
focal_plane_y = 1003.76
focal_plane_offset = 1.0

grating1 = OpticalElement(
    name="Grating 1", optic_type="Grating",
    x_center=12.7, y_center=focal_plane_y + focal_plane_offset, 
    orientation_angle=-90.0,   
    clear_aperture=45, diameter=50.8,
    groove_density=75.0, diffraction_order=-1, material="Gold"
)

grating2 = OpticalElement(
    name="Grating 2", optic_type="Grating",
    x_center=-10.0, y_center=(focal_plane_y - 25.0) + focal_plane_offset, 
    orientation_angle=0,   
    clear_aperture=45, diameter=50.8,
    groove_density=25.0, diffraction_order=1, material="Gold" 
)

manager.add_element(lens1)
manager.add_element(lens2)
manager.add_element(grating1)
manager.add_element(grating2)


In [20]:
# 1. Optimize Lens 1 Collimation
opt_L1_y = optimization.optimize_lens1_collimation(manager, guess_y=250.0, search_window=5.0)

# 2. Lock Lens 2 to exact 4f distance (2 * 250mm)
lens2 = next(el for el in manager.elements if el.name == "Lens 2")
lens2.y_center = opt_L1_y + 500.0

# 3. Find exact focal plane
opt_focus_y = optimization.optimize_focal_plane(manager, guess_y=lens2.y_center + 248.0, search_window=10.0)

# 4. Optional: Optimize gratings for higher-order residuals
optimization.optimize_compressor_gratings(manager, focal_plane_base=opt_focus_y)

--- Optimizing Lens 1 for Collimation ---
--- Smart Source Generation (Target: Lens 1) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 12.00mm to 13.00mm (Extent: 1.00mm)
   Step 2: Optimizing angles for 1 sources...
     Src 1: X=12.0mm | Angles=[87.2°, 92.4°] (30 rays)
   Done. Generated 30 optimized rays.
--- Smart Source Generation (Target: Lens 1) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 12.00mm to 13.00mm (Extent: 1.00mm)
   Step 2: Optimizing angles for 1 sources...
     Src 1: X=12.0mm | Angles=[87.2°, 92.4°] (30 rays)
   Done. Generated 30 optimized rays.
--- Smart Source Generation (Target: Lens 1) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 12.00mm to 13.00mm (Extent: 1.00mm)
   Step 2: Optimizing angles for 1 sources...
     Src 1: X=12.0mm | Angles=[87.2°, 92.4°] (30 rays)
   Done. Generated 30 optimized rays.
--- Smart Source Generation (Target: Lens 1) ---
   Step 1: Finding valid

     Src 1: X=12.0mm | Angles=[87.2°, 92.4°] (30 rays)
   Done. Generated 30 optimized rays.
--- Smart Source Generation (Target: Lens 1) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 12.00mm to 13.00mm (Extent: 1.00mm)
   Step 2: Optimizing angles for 1 sources...


KeyboardInterrupt: 

In [ ]:
tracer = RayTracer(manager)

# --- SMART GENERATION ---
# Automatically finds the valid grating region and optimizes angles
tracer.generate_smart_spr_source(
    n_sources=11,                # Number of distinct source points you want
    rays_per_source=20,         # Rays per point (uniformly distributed in valid cone)
    target_optic_name="Lens 2", # The optic that defines "success" (can be OAP/Mirror too)
    grating_search_bounds=(0, 25.4), # Max physical length of grating to scan
    acceptance_angle_range=(70, 110), 
    grating_period=10.0,
    beam_energy=0.99
)    

# 3. Run Simulation
print("Running Simulation...")
for t in np.arange(0, 5500, 50.0):
    tracer.run_time_step(t, 50.0)
tracer._sync_to_dataframe()

# 4. VISUALIZATION (Now handled entirely by the class)
# This automatically handles the slider, the figure creation, 
# and preserving your zoom level when switching sources.
Draw.interactive_session(
manager, 
tracer, 
show_curvature=False, 
show_skeleton=True,       # <--- You can toggle these easily now
draw_beam_arrow=True,
show_intersection_points=False
)
